# 1. HelmholtzOperator

`HelmholtzOperator` is the forward/adjoint operator for single‐frequency Helmholtz full‐waveform inversion, encapsulating the internal `HelmholtzSolver`.

---

### Functionality
- **forward(m)**
  Simulate Tx × Rx frequency‐domain data \(F(m)\).
- **solve(src, adjoint=False)**
  Solve the forward or adjoint Helmholtz equation for an arbitrary source array.
- **get_field(key)**
  Read‐only access to cached fields:
  - `WF` (forward wavefield)
  - `VSRC` (virtual‐source)
  - `scaling` (scaling coefficients to simData)
  - `obs_data` (observed data)
  - `PML`, `V`, `freq` (PML, Complex velocity, frequency)

---

### Constructor Parameters

| Parameter    | Type                | Description                                          |
|--------------|---------------------|------------------------------------------------------|
| `data`       | `AcquisitionData`   | Measured data + acquisition geometry                 |
| `f_idx`      | `int`               | Frequency index (uses `data.freqs[f_idx]`)           |
| `img_geo`    | `ImageGeometry`     | Imaging‐grid geometry                                |
| `sign_conv`  | `int`               | Helmholtz sign convention (+1 or −1)                 |
| `a0`         | `float`             | PML absorption coefficient                           |
| `L_PML`      | `float`             | PML thickness                                        |
| `canUseGPU`  | `bool`, optional    | Whether to use GPU acceleration (default: False)     |

---

### Main Methods

```python
from FullWaveUST.optimization.operator.helmholtz import HelmholtzOperator

# Create the operator
op = HelmholtzOperator(
    data, f_idx, img_geo,
    sign_conv=-1, a0=10.0, L_PML=9e-3
)

# 1) Simulate frequency‐domain data F(m)
F = op.forward(m)  # m: (ny, nx) complex slowness → F: (Tx, Rx)

# 2) Solve wave equation (forward or adjoint)
wavefield, _ = op.solve(src, adjoint=True)

# 3) Inspect cached fields
PML      = op.get_field('PML')
WF       = op.get_field('WF')
obs_data = op.get_field('obs_data')
```

### Cache Behavior
•	Building cache Occurs on first forward(m) call or whenever m changes.
•	Cached attributes: `WF`, `VSRC`, `scaling`, `obs_data`, `PML`, `V`, `freq`.

---

## 2. AdjointHelmholtzGrad

`AdjointHelmholtzGrad` computes the adjoint‐state gradient for single‐frequency Helmholtz FWI.

---

### Functionality
- Computes the real‐valued gradient for a given model and residual(adjoint source).
- Allows user‐supplied ∂H(s)/∂s `deriv_fn(m, op)`.

---

### Constructor Parameters

| Parameter   | Type                                                              | Description                                                                                  |
|-------------|-------------------------------------------------------------------|----------------------------------------------------------------------------------------------|
| `op`        | `HelmholtzOperator`                                              | Forward/adjoint operator                                                                     |
| `kernal_fn` | `Callable[[m, op], ndarray]`, optional                           | Function returning gradient kernel (K(x) of shape (ny, nx); default: K(x) = 8π² f² · PML / V |

---

### Main Method

```python
from FullWaveUST.optimization.gradient.adjoint_helmholtz import AdjointHelmholtzGrad

# Create the gradient evaluator
grad_eval = AdjointHelmholtzGrad(op, deriv_fn=my_kernel)

# Compute residual r = F(m) - d
r = op.forward(m) - op.get_field('obs_data')

# Compute gradient g(x)
g = grad_eval.gradient(m, q=r * weight)  # returns real (ny, nx)

---

## 3. NonlinearLS

`NonlinearLS` wraps the weighted L² misfit:

$$
\Phi(m) = \tfrac12 \lVert w \,\odot\, (F(m) - d)\rVert_2^2
$$

---

### Functionality
- Computes the data residual, scalar objective, and gradient.
- Caches forward results to avoid redundant solves.
- Gradient is computed using `AdjointHelmholtzGrad`, given a residual \(r\).
---

### Constructor Parameters

| Parameter   | Type                                            | Description                                  |
|-------------|-------------------------------------------------|----------------------------------------------|
| `op`        | `HelmholtzOperator`                            | Forward operator                            |
| `grad_eval` | `GradientEvaluator` (e.g. `AdjointHelmholtzGrad`) | Gradient evaluator                          |
| `weight`    | `float` or `ndarray`, optional, default `1.0`   | Residual weighting; must be positive        |

---

### Main Methods

```python
from FullWaveUST.optimization.function.nonlinear_ls import NonlinearLS

fun = NonlinearLS(op, grad_eval=grad_eval, weight=1.0)

# 1) Residual
r = fun.residual(m)      # complex (Tx,Rx)

# 2) Objective value
phi = fun.value(m)       # scalar

# 3) Gradient
g = fun.gradient(m)      # real ndarray (ny, nx)

---

## 4. CG (Conjugate Gradient)

`CG` implements non‐linear conjugate gradient for Helmholtz FWI.

---

### Functionality
- **Direction update** using Polak–Ribiere with Fletcher–Reeves bound.
- **Closed‐form step size** by first‐order Taylor expansion.
- **Mode switching**: update real part, imaginary part, or both.
- **Online recorder**: stores snapshots of velocity, attenuation, gradient, and search direction at each iteration.
- **Visualization hook**: accepts `viz.update(slow, grad, search_dir)` for live plotting.

---

### Constructor Parameters

| Parameter | Type                                      | Description                                  |
|-----------|-------------------------------------------|----------------------------------------------|
| `mode`    | `{'full','real','imag'}`, default `'full'` | Which part of \(m\) to update               |
| `c1`      | `float`, default `1e-4`                   | Armijo parameter   |
| `shrink`  | `float`, default `0.5`                    | Backtracking factor                |
| `max_ls`  | `int`, default `20`                       | Max line‐search steps            |

---

### Main Methods

```python
from FullWaveUST.optimization.algorithm.cg import CG

cg = CG(mode='real', c1=1e-4, shrink=0.5, max_ls=20)

slow_final = cg.solve(
    fun,           # NonlinearLS instance
    slow_data,     # ImageData initial model
    n_iter=10,     # number of iterations
    mode='real',   # update only real part
    viz=viz,       # InversionVisualizer instance
    do_print_time=True
)

records = cg.get_record()
# keys: 'vel','atten','grad','search', each is shape (ny, nx, n_iter)

cg.reset()  # clear state and recorder